# Conexión a Supabase con SQLModel

Para conectar SQLModel a Supabase necesitás:

1. Instalar el driver de PostgreSQL: `uv add psycopg2-binary`
2. Obtener tu password de Supabase
3. Configurar la variable de entorno `SUPABASE_PASSWORD`

## Pasos:

```bash
# En terminal
export SUPABASE_PASSWORD="tu_password_aqui"
```

O crear un archivo `.env`:
```
SUPABASE_PASSWORD=tu_password_aqui
```


# SQLModel
ORM Moderno basado en Pydantic y SQLAlchemy



In [ ]:
!uv add sqlmodel

In [1]:
from sqlmodel import SQLModel, Field
import json

class Persona(SQLModel):
    nombre: str
    edad: int

p = Persona(nombre="Ana", edad=28)
print(p)
print(p.model_dump())
print(p.model_dump_json(indent=2))
print(json.dumps(p.model_json_schema(), indent=2))



nombre='Ana' edad=28
{'nombre': 'Ana', 'edad': 28}
{
  "nombre": "Ana",
  "edad": 28
}
{
  "properties": {
    "nombre": {
      "title": "Nombre",
      "type": "string"
    },
    "edad": {
      "title": "Edad",
      "type": "integer"
    }
  },
  "required": [
    "nombre",
    "edad"
  ],
  "title": "Persona",
  "type": "object"
}


In [2]:
from sqlmodel import create_engine, SQLModel, Field

class Usuario(SQLModel, table=True):
    id: int | None = Field(default=None, primary_key=True)
    nombre: str
    email: str  
    telefono: str | None = None
    
usuario = Usuario(nombre="juan", email="juan@gmail.com")
print(usuario)

engine = create_engine("sqlite:///database.db")
SQLModel.metadata.create_all(engine)

nombre='juan' email='juan@gmail.com' id=None telefono=None


In [3]:
from sqlmodel import SQLModel, Field, Session, select

with Session(engine) as db:
    usuario = Usuario(nombre="maria", email="maria@gmail.com")
    db.add(usuario)
    db.commit()
    db.refresh(usuario)
    print(usuario)

telefono=None id=1 nombre='maria' email='maria@gmail.com'


In [4]:
f = open("texto.txt", "w")
f.write("Hola, mundo!")
f.close()

with open("texto2.txt", "w") as f:
    f.write("Hola, mundo con contexto!")

In [5]:
s = Session(engine)
s.add(Usuario(nombre="carlos", email="carlos@gmail.com"))
s.add(Usuario(nombre="luisa", email="luisa@gmail.com"))
s.commit()

In [6]:
s = Session(engine)
statement = select(Usuario).where(Usuario.nombre == "maria")
print(statement)
for u in s.exec(statement):
    print(u)

SELECT usuario.id, usuario.nombre, usuario.email, usuario.telefono 
FROM usuario 
WHERE usuario.nombre = :nombre_1
telefono=None id=1 nombre='maria' email='maria@gmail.com'


In [16]:
with Session(engine) as db:
    usuarios = db.exec(select(Usuario)).all()
    for usuario in usuarios:
        print(usuario)

nombre='maria' email='maria@gmail.com' id=1 telefono=None
nombre='maria' email='maria@gmail.com' id=2 telefono=None
nombre='carlos' email='carlos@gmail.com' id=3 telefono=None
nombre='luisa' email='luisa@gmail.com' id=4 telefono=None
nombre='carlos' email='carlos@gmail.com' id=5 telefono=None
nombre='luisa' email='luisa@gmail.com' id=6 telefono=None


In [ ]:
with Session(engine) as db:
    # Usando get() - más directo para obtener por clave primaria
    usuario = db.get(Usuario, 4)
    print(f"Usuario con ID 4: {usuario}")

    # Comparación con el método anterior (select + where + first)
    usuario_select = db.exec(select(Usuario).where(Usuario.id == 4)).first()
    print(f"Usuario con select: {usuario_select}")

    # Intentar obtener un usuario que no existe
    usuario_inexistente = db.get(Usuario, 999)
    print(f"Usuario inexistente: {usuario_inexistente}")
    


nombre='luisa' email='luisa@gmail.com' id=4 telefono=None


In [8]:
from contextlib import contextmanager

@contextmanager
def registrar():
    session = Session(engine)
    try:
        yield session   
        session.commit()    
        print("Commit exitoso")
    except Exception as e:
        session.rollback()
        print(f"Error, se hizo rollback: {e}")
        raise
    finally:
        session.close()
            
@contextmanager
def leer(selector):
    with Session(engine) as session:
        try:
            result = session.exec(selector)
            yield result
        except Exception as e:
            print(f"Error al leer: {e}")
            raise
        
# Verificar que se guardó
with leer(select(Usuario).where(Usuario.nombre == "pedro")) as db:
    pedro = db.first()
    print(f"Usuario guardado: {pedro}")

print("Fin del programa")

Usuario guardado: None
Fin del programa


## Modelo de Factura con Items

Vamos a crear un sistema de facturación con múltiples items usando relaciones entre tablas.

In [6]:
from sqlmodel import SQLModel, Field, Relationship, create_engine, Session, select
from datetime import datetime
from typing import Optional

# Modelo para la Factura (cabecera)
class Factura(SQLModel, table=True):
    __tablename__ = "facturas"
    
    id: Optional[int] = Field(default=None, primary_key=True)
    numero: str = Field(index=True, unique=True)
    cliente: str
    fecha: datetime = Field(default_factory=datetime.now)
    observaciones: Optional[str] = None
    
    # Relación con los items (uno a muchos)
    items: list["ItemFactura"] = Relationship(back_populates="factura")
    
    @property
    def total(self) -> float:
        """Calcula el total de la factura sumando todos los items"""
        return sum(item.subtotal for item in self.items)
    
    def __repr__(self):
        return f"Factura(numero={self.numero}, cliente={self.cliente}, total=${self.total:.2f})"


# Modelo para cada Item de la Factura
class ItemFactura(SQLModel, table=True):
    __tablename__ = "items_factura"
    
    id: Optional[int] = Field(default=None, primary_key=True)
    factura_id: int = Field(foreign_key="facturas.id")
    producto: str
    descripcion: Optional[str] = None
    cantidad: int
    precio_unitario: float
    
    # Relación con la factura
    factura: Optional[Factura] = Relationship(back_populates="items")
    
    @property
    def subtotal(self) -> float:
        """Calcula el subtotal del item"""
        return self.cantidad * self.precio_unitario
    
    def __repr__(self):
        return f"Item(producto={self.producto}, cantidad={self.cantidad}, subtotal=${self.subtotal:.2f})"

print("✅ Modelos de Factura e ItemFactura creados")

✅ Modelos de Factura e ItemFactura creados


In [7]:
from sqlmodel import create_engine, SQLModel
# Crear la base de datos y las tablas
engine_factura = create_engine("sqlite:///facturas.db")
SQLModel.metadata.create_all(engine_factura)

print("✅ Base de datos 'facturas.db' creada con las tablas")

✅ Base de datos 'facturas.db' creada con las tablas


### Crear una factura con items

Ejemplo de cómo crear una factura completa con múltiples items.

In [ ]:
with Session(engine_factura) as db:
    # Crear una nueva factura
    factura = Factura(
        numero="FAC-001",
        cliente="Acme Corporation",
        observaciones="Pago en 30 días"
    )
    
    # Agregar items a la factura
    item1 = ItemFactura(
        producto="Laptop Dell XPS 15",
        descripcion="Intel i7, 16GB RAM, 512GB SSD",
        cantidad=2,
        precio_unitario=1500.00,
        factura=factura
    ) # type: ignore
    
    item2 = ItemFactura(
        producto="Mouse Logitech MX Master",
        descripcion="Ergonómico, inalámbrico",
        cantidad=2,
        precio_unitario=99.99,
        factura=factura
    ) # type: ignore
    
    item3 = ItemFactura(
        producto="Teclado mecánico",
        descripcion="Switches Cherry MX Blue",
        cantidad=2,
        precio_unitario=150.00,
        factura=factura
    ) # pyright: ignore[reportCallIssue]
    
    # Agregar la factura a la sesión (los items se agregan automáticamente)
    db.add(factura)
    db.commit()
    db.refresh(factura)
    
    print(f"✅ Factura creada: {factura}")
    print(f"\nItems:")
    for item in factura.items:
        print(f"  - {item}")

✅ Factura creada: numero='FAC-001' cliente='Acme Corporation' observaciones='Pago en 30 días' id=1 fecha=datetime.datetime(2025, 10, 21, 3, 36, 17, 609422)

Items:
  - producto='Laptop Dell XPS 15' factura_id=1 cantidad=2 descripcion='Intel i7, 16GB RAM, 512GB SSD' id=1 precio_unitario=1500.0
  - producto='Mouse Logitech MX Master' factura_id=1 cantidad=2 descripcion='Ergonómico, inalámbrico' id=2 precio_unitario=99.99
  - producto='Teclado mecánico' factura_id=1 cantidad=2 descripcion='Switches Cherry MX Blue' id=3 precio_unitario=150.0


In [14]:
# Crear otra factura
with Session(engine_factura) as db:
    factura2 = Factura(
        numero="FAC-002",
        cliente="TechStart Solutions"
    )
    
    items = [
        ItemFactura(
            producto="Monitor 27\" 4K",
            cantidad=3,
            precio_unitario=450.00,
            factura=factura2
        ),
        ItemFactura(
            producto="Webcam HD",
            cantidad=3,
            precio_unitario=89.99,
            factura=factura2
        ),
        ItemFactura(
            producto="Auriculares con micrófono",
            descripcion="Cancelación de ruido",
            cantidad=3,
            precio_unitario=120.00,
            factura=factura2
        ),
    ]
    
    db.add(factura2)
    db.commit()
    db.refresh(factura2)
    
    print(f"✅ Segunda factura creada: {factura2}")
    print(f"\nItems:")
    for item in factura2.items:
        print(f"  - {item}")

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: facturas.numero
[SQL: INSERT INTO facturas (numero, cliente, fecha, observaciones) VALUES (?, ?, ?, ?)]
[parameters: ('FAC-002', 'TechStart Solutions', '2025-10-21 03:37:52.545599', None)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

### Consultar facturas

Veamos cómo recuperar y mostrar las facturas con sus items.

In [10]:
# Listar todas las facturas
with Session(engine_factura) as db:
    facturas = db.exec(select(Factura)).all()
    
    print("📋 LISTADO DE FACTURAS")
    print("=" * 80)
    
    for factura in facturas:
        print(f"\n{factura}")
        print(f"Fecha: {factura.fecha.strftime('%d/%m/%Y %H:%M')}")
        if factura.observaciones:
            print(f"Observaciones: {factura.observaciones}")
        print(f"\nDetalle de items:")
        print(f"{'Producto':<30} {'Cant':>6} {'P.Unit':>10} {'Subtotal':>12}")
        print("-" * 80)
        
        for item in factura.items:
            print(f"{item.producto:<30} {item.cantidad:>6} ${item.precio_unitario:>9.2f} ${item.subtotal:>11.2f}")
        
        print("-" * 80)
        print(f"{'TOTAL':<48} ${factura.total:>11.2f}")
        print("=" * 80)

📋 LISTADO DE FACTURAS

numero='FAC-001' cliente='Acme Corporation' observaciones='Pago en 30 días' id=1 fecha=datetime.datetime(2025, 10, 21, 3, 36, 17, 609422)
Fecha: 21/10/2025 03:36
Observaciones: Pago en 30 días

Detalle de items:
Producto                         Cant     P.Unit     Subtotal
--------------------------------------------------------------------------------
Laptop Dell XPS 15                  2 $  1500.00 $    3000.00
Mouse Logitech MX Master            2 $    99.99 $     199.98
Teclado mecánico                    2 $   150.00 $     300.00
--------------------------------------------------------------------------------
TOTAL                                            $    3499.98

numero='FAC-002' cliente='TechStart Solutions' observaciones=None id=2 fecha=datetime.datetime(2025, 10, 21, 3, 36, 23, 953681)
Fecha: 21/10/2025 03:36

Detalle de items:
Producto                         Cant     P.Unit     Subtotal
-----------------------------------------------------------

In [11]:
# Buscar una factura específica por número
with Session(engine_factura) as db:
    factura = db.exec(
        select(Factura).where(Factura.numero == "FAC-001")
    ).first()
    
    if factura:
        print(f"🔍 Factura encontrada: {factura.numero}")
        print(f"Cliente: {factura.cliente}")
        print(f"Total: ${factura.total:.2f}")
        print(f"Cantidad de items: {len(factura.items)}")

🔍 Factura encontrada: FAC-001
Cliente: Acme Corporation
Total: $3499.98
Cantidad de items: 3


### Operaciones avanzadas

Actualizar items, agregar nuevos items a una factura existente, y calcular estadísticas.

In [12]:
# Agregar un nuevo item a una factura existente
with Session(engine_factura) as db:
    factura = db.exec(
        select(Factura).where(Factura.numero == "FAC-001")
    ).first()
    
    if factura:
        nuevo_item = ItemFactura(
            producto="Cable HDMI 2.1",
            descripcion="2 metros, 4K@120Hz",
            cantidad=5,
            precio_unitario=25.00,
            factura=factura
        )
        
        db.add(nuevo_item)
        db.commit()
        db.refresh(factura)
        
        print(f"✅ Nuevo item agregado a la factura {factura.numero}")
        print(f"Nuevo total: ${factura.total:.2f}")

✅ Nuevo item agregado a la factura FAC-001
Nuevo total: $3624.98


In [15]:
# Estadísticas generales
with Session(engine_factura) as db:
    facturas = db.exec(select(Factura)).all()
    items = db.exec(select(ItemFactura)).all()
    
    total_facturado = sum(f.total for f in facturas)
    cantidad_facturas = len(facturas)
    cantidad_items = len(items)
    promedio_por_factura = total_facturado / cantidad_facturas if cantidad_facturas > 0 else 0
    
    print("📊 ESTADÍSTICAS DEL SISTEMA")
    print("=" * 60)
    print(f"Total de facturas: {cantidad_facturas}")
    print(f"Total de items vendidos: {cantidad_items}")
    print(f"Total facturado: ${total_facturado:.2f}")
    print(f"Promedio por factura: ${promedio_por_factura:.2f}")
    print("=" * 60)

📊 ESTADÍSTICAS DEL SISTEMA
Total de facturas: 2
Total de items vendidos: 7
Total facturado: $5604.95
Promedio por factura: $2802.47


In [ ]:
from contextlib import contextmanager

class ContactoBase(SQLModel):
    nombre: str
    email: str
    telefono: Optional[str] = None

class Contacto(ContactoBase, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    
class ContactoCreate(ContactoBase):
    pass

class ContactoUpdate(SQLModel):
    nombre: Optional[str] = None
    email: Optional[str] = None
    telefono: Optional[str] = None
    
class ContactoRead(ContactoBase):
    id: int

@contextmanager
def get_session():
    """
    Generador de contexto para obtener una sesión de base de datos.
    Hace commit automáticamente al finalizar si no hay errores.
    Hace rollback si hay una excepción.
    """
    session = Session(engine_factura)
    try:
        yield session
        session.commit()  # Commit automático al finalizar
    except Exception as e:
        session.rollback()  # Rollback si hay error
        raise
    finally:
        session.close()
            
class ServicioContacto:
    """Servicio para gestionar contactos usando el patrón Repository"""
    
    def crear_contacto(self, contacto_create: ContactoCreate) -> Contacto:
        """Crea un nuevo contacto en la base de datos"""
        with get_session() as session:
            nuevo_contacto = Contacto(**contacto_create.model_dump())
            session.add(nuevo_contacto)
            # No necesitamos commit aquí, get_session lo hace automáticamente
            session.flush()  # Para obtener el ID asignado
            session.refresh(nuevo_contacto)
            return nuevo_contacto

    def obtener_contacto(self, contacto_id: int) -> Optional[Contacto]:
        """Obtiene un contacto por su ID"""
        with get_session() as session:
            return session.get(Contacto, contacto_id)

    def listar_contactos(self) -> list[Contacto]:
        """Lista todos los contactos"""
        with get_session() as session:
            return session.exec(select(Contacto)).all()

    def actualizar_contacto(self, contacto_id: int, contacto_update: ContactoUpdate) -> Optional[Contacto]:
        """Actualiza un contacto existente"""
        with get_session() as session:
            contacto = session.get(Contacto, contacto_id)
            if not contacto:
                return None
            
            # Actualizar solo los campos que fueron proporcionados
            for key, value in contacto_update.model_dump(exclude_unset=True).items():
                setattr(contacto, key, value)

            session.add(contacto)
            # No necesitamos commit aquí, get_session lo hace automáticamente
            session.flush()
            session.refresh(contacto)
            return contacto

    def eliminar_contacto(self, contacto_id: int) -> bool:
        """Elimina un contacto por su ID"""
        with get_session() as session:
            contacto = session.get(Contacto, contacto_id)
            if not contacto:
                return False
            session.delete(contacto)
            # No necesitamos commit aquí, get_session lo hace automáticamente
            return True    

print("✅ Modelos y ServicioContacto refactorizados con get_session que hace commit automático")

### Context Manager con Commit Automático

El `get_session()` ahora funciona como un context manager completo:

✅ **Commit automático**: Al salir del bloque `with` sin errores  
✅ **Rollback automático**: Si ocurre una excepción  
✅ **Cierre garantizado**: La sesión siempre se cierra en el `finally`  

Esto simplifica el código en los métodos del servicio:
- ❌ Ya no necesitamos `session.commit()` en cada método
- ✅ Usamos `session.flush()` para obtener IDs antes del commit
- ✅ Las transacciones son automáticas y seguras

### Ejemplo de uso del ServicioContacto

Vamos a probar todas las operaciones CRUD del servicio refactorizado.

In [ ]:
# Crear instancia del servicio (ya no necesita sesión en el constructor)
servicio = ServicioContacto()

# 1. Crear contactos
print("1️⃣ Creando contactos...")
contacto1 = servicio.crear_contacto(
    ContactoCreate(
        nombre="Juan Pérez",
        email="juan@example.com",
        telefono="555-1234"
    )
)
print(f"   ✅ Creado: {contacto1}")

contacto2 = servicio.crear_contacto(
    ContactoCreate(
        nombre="María González",
        email="maria@example.com"
    )
)
print(f"   ✅ Creado: {contacto2}")

# 2. Listar todos los contactos
print("\n2️⃣ Listando todos los contactos...")
contactos = servicio.listar_contactos()
for contacto in contactos:
    print(f"   - ID: {contacto.id} | {contacto.nombre} | {contacto.email} | Tel: {contacto.telefono or 'N/A'}")

# 3. Obtener un contacto específico
print(f"\n3️⃣ Obteniendo contacto con ID {contacto1.id}...")
contacto_encontrado = servicio.obtener_contacto(contacto1.id)
if contacto_encontrado:
    print(f"   ✅ Encontrado: {contacto_encontrado}")

# 4. Actualizar un contacto
print(f"\n4️⃣ Actualizando contacto con ID {contacto2.id}...")
contacto_actualizado = servicio.actualizar_contacto(
    contacto2.id,
    ContactoUpdate(telefono="555-5678")
)
if contacto_actualizado:
    print(f"   ✅ Actualizado: {contacto_actualizado}")

# 5. Eliminar un contacto
print(f"\n5️⃣ Eliminando contacto con ID {contacto1.id}...")
eliminado = servicio.eliminar_contacto(contacto1.id)
print(f"   {'✅ Eliminado' if eliminado else '❌ No se pudo eliminar'}")

# 6. Verificar la lista final
print("\n6️⃣ Lista final de contactos:")
contactos_finales = servicio.listar_contactos()
for contacto in contactos_finales:
    print(f"   - ID: {contacto.id} | {contacto.nombre} | {contacto.email} | Tel: {contacto.telefono or 'N/A'}")